# Purpose

Now that we generated a year's worth of trasaction data, we can start to generate recommendations for our customers.

Since we don't have any data related to customer preference or rating for particular grocery items (let's assume that our shop is offline only and getting this information is costly), we are bound to being able to give non-personalised recommendations to customers.

For example, we can explore the data to discover items frequently bought together. It doesn't recommend items specefic to customers, but it's a good starting point. An example would be recommending bread when a customer buys both milk and butter, since milk, butter, and bread are often purchased together.

In [1]:
import pandas as pd
import sqlite3
from sqlite3 import Error

In [2]:
# Functions
def create_connection(db_file):
    """
    Create a database connection to a SQLite database.
    
    The db_file should be the path to a .db file.
    
    If you pass the file name as :memory: to the connect() function 
    of the sqlite3 module, it will create a new database that resides 
    in the memory (RAM) instead of a database file on disk.
    
    Returns a connection object.
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    
    except Error as e:
        print(e)

    return conn

Happily for us, we can use pandas to read data from SQL databases (see https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html). So let's do that!

In [3]:
database_file = '../databases/groceries.db'
conn = create_connection(database_file)

bills = pd.read_sql(
    'select * from billings;',
    conn
)
display(bills.head())

items = pd.read_sql(
    'select * from items;',
    conn
)
display(items.head())

,id,timestamp,customer_id,item
0,859724875,1926288046,958420868,36088
1,859724875,1926288046,958420868,7244
2,2768611031,1926288019,4036240255,5651
3,2768611031,1926288019,4036240255,5049
4,2768611031,1926288019,4036240255,64051


,id,product_name,type,price
0,32,Homestyle Straight Cut Oven Chips 950G,frozen-food,1.4
1,552,Pampers Baby Dry Pants Essential Pack Size 6 2...,baby,7.0
2,885,Fred & Flo 60 Sensitive Toilet Wipes,baby,0.7
3,1584,Colgate Total Original Care Toothpaste 125Ml,health-and-beauty,2.5
4,1614,Carte D'or Vanilla Ice Cream Dessert 1L,frozen-food,3.5


Ok, so now that we have the data, the first step is finding out what the most popular item we have is. We can do this by using pandas' `.value_counts()` of a Series (column).

In [4]:
display(bills['item'].value_counts())

52381    269
39686    263
43094    259
57458    259
16744    258
        ... 
38204    146
39073    134
61457     95
42932     78
51708     69
Name: item, Length: 240, dtype: int64

Great, but we don't know what the item is... So let's merge the two dataframes to get the name of the item.

In [5]:
display(
    pd.merge(
        bills['item'].value_counts(),
        items,
        left_index=True,
        right_on='id'
    ).drop(['id', 'price'], axis=1).rename(
        columns={'item': 'times_purchased'}
    )
)

,times_purchased,product_name,type
199,269,100% Pressed Apple Juice 1 Litre,drinks
145,263,Small Bananas 6 Pack,fresh-food
161,259,Pepsi Max Cola 2 Litre Bottle,drinks
217,259,Chickpeas In Water 400G,food-cupboard
56,258,Original Source Mint Shower Gel 250Ml,health-and-beauty
...,...,...,...
138,146,Pepsi Max 24 X 330Ml,drinks
141,134,Felix As Good As It Looks Cat Food Ocean Feast...,pets
230,95,Gold Leaf Jps 50G,home-and-ents
160,78,Golden Virginia Original 50G,home-and-ents


The top and bottom items seem to suggest that the bulk of our customers pay modest attention to their physical health (with apple juice and bananas being the top 2 items purchased in the year, and tobacco being the lowest purchases). They also, for some reason, really like chickpeas (are they hummus fans?), and a few of them have cats!

Next, we find items most commonly bought together. Since we don't have any rating or review attached to each item, this is the best way we have to make recommendations. To start, we will record each time a pair of items is seen together in a given transaction, then create a look-up table for the permutation of pairs.

In [6]:
from itertools import permutations

In [13]:
def create_pairs(x):
    """Creates pairs of items for every transaction.
    
    Returns a DataFrame for convenience of analysis."""
    pairs = pd.DataFrame(
        # Casts iterable to a list
        list(
            # Create a permutation iterable with 2 values
            permutations(
                x['item'].values,
                # If A and B are in the data, returns
                # [A, B] and [B, A].
                2
            )
        ),
        columns=['item_a', 'item_b']
    )
    return pairs

In [19]:
# Testing
item_pairs = bills.head(20).groupby('id').apply(
    create_pairs)
display(item_pairs)

item_a  item_b
id                          
55789631   0   19394   33533
           1   19394   17407
           2   19394   57741
           3   19394   14679
           4   19394   16744
...              ...     ...
2768611031 1    5651   64051
           2    5049    5651
           3    5049   64051
           4   64051    5651
           5   64051    5049

[106 rows x 2 columns]

Ok, now we just need to apply it to the rest of the transaction. We will also drop the index, since we don't need it anymore.

In [22]:
%%time 
item_pairs = bills.groupby('id').apply(
    create_pairs).reset_index(drop=True)
display(item_pairs)

,item_a,item_b
0,21511,41371
1,21511,25512
2,21511,46361
3,21511,13484
4,21511,64049
...,...,...
1363101,60756,36126
1363102,39584,46225
1363103,46225,39584
1363104,53728,52022


CPU times: user 2.59 s, sys: 76.3 ms, total: 2.67 s
Wall time: 2.68 s


So now we group the data and count the number of times each pair is found in our dataframe!

In [35]:
pair_counts = item_pairs.groupby(
    ['item_a', 'item_b']
).size().to_frame(name = 'size').reset_index()
display(pair_counts.sort_values('size', ascending=False).head())
print(pair_counts.shape)

,item_a,item_b,size
37817,42234,37577,52
33037,37577,42234,52
27097,30026,57458,52
52192,57458,30026,52
27068,30026,49742,51


(57600, 3)


Again, we merge two dataframes to get the names of the items!

In [47]:
pair_counts_wnames = pd.merge(
    pd.merge(
        pair_counts,
        items[['id', 'product_name']],
        left_on='item_a',
        right_on='id'
    ).drop(['id'], axis=1).rename(
        columns={'product_name': 'product_name_a'}
    ),
    items[['id', 'product_name']],
    left_on='item_b',
    right_on='id'
).drop(['id'], axis=1).rename(
    columns={'product_name': 'product_name_b'}
)
display(pair_counts_wnames.sort_values('size', ascending=False).head(10))

,item_a,item_b,size,product_name_a,product_name_b
37817,37577,42234,52,Pampers Sensitive Baby Wipes 52 Pack,Indian Tonic Water 1L
33037,42234,37577,52,Indian Tonic Water 1L,Pampers Sensitive Baby Wipes 52 Pack
27097,57458,30026,52,Chickpeas In Water 400G,Pride Coconut Milk 400Ml
52192,30026,57458,52,Pride Coconut Milk 400Ml,Chickpeas In Water 400G
27068,49742,30026,51,8 Plain Tortilla Wraps,Pride Coconut Milk 400Ml
45232,30026,49742,51,Pride Coconut Milk 400Ml,8 Plain Tortilla Wraps
56961,24056,64051,51,10 Omega 3 Fish Fingers 300G,Hash Browns 750G
19677,64051,24056,51,Hash Browns 750G,10 Omega 3 Fish Fingers 300G
38032,30026,42405,50,Pride Coconut Milk 400Ml,Kp Skips Prawn Cocktail Crisps 6 X 13.1G
5202,43391,6133,50,Cadbury Dairy Milk Chocolate Bar 110G,Diet Coke 2L


Well, our data suggests that people who have bought baby wipes also bought tonic water (perhaps for the gin?). With this data, we can now create very simple recommendations for customers: customers who purchased tonic water have also purchased baby wipes in the same shopping trip!